# Simulation 2: Dependence on recording length

## Sampling from an "intermediately sized" LDS, fitting at different observation ratios

In [ ]:

from __future__ import division
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

from pybasicbayes.distributions import Regression
from pybasicbayes.util.text import progprint_xrange
from autoregressive.distributions import AutoRegression

import pyximport
pyximport.install()
import os
os.chdir('../pylds')
from models import LDS, DefaultLDS
from distributions import Regression_diag, AutoRegression_input
from obs_scheme import ObservationScheme
from user_util import gen_pars, rand_rotation_matrix, init_LDS_model, collect_LDS_stats
#npr.seed(0)

def update(model):
    model.EM_step()
    return model.log_likelihood()                    

#########################
#  set some parameters  #
#########################

p = 30
T = 1000

m = 500
num_restart = 10

n = 5

num_restart = 1

eps = np.log(1.01)

def nrmls(a):
    a = a/a.sum()
    return a

prots = [ np.around( nrmls(np.array([ 33., 33., 33.])).cumsum() * T ),          
          np.around( nrmls(np.array([ 20., 20., 60.])).cumsum() * T ),
          np.around( nrmls(np.array([ 10., 10., 80.])).cumsum() * T ),
         
          np.around( nrmls(np.array([ 11., 55., 33.])).cumsum() * T ),
          np.around( nrmls(np.array([ 22., 44., 33.])).cumsum() * T ),
          np.around( nrmls(np.array([ 44., 22., 33.])).cumsum() * T ),
          np.around( nrmls(np.array([ 55., 11., 33.])).cumsum() * T )         
        ]
for i in range(len(prots)):
    prots[i] = prots[i].astype(np.int)

for k in range(len(prots)):
    for i in range(num_restart):

        pars_true, _ = gen_pars(n, p, u_dim=0, 
                 pars_in=None, 
                 obs_scheme=None,
                 gen_A='diagonal', lts=np.linspace(0.95, 0.98, n),
                 gen_B='random', 
                 gen_Q='identity', 
                 gen_mu0='random', 
                 gen_V0='identity', 
                 gen_C='random', 
                 gen_d='scaled', 
                 gen_R='fraction',
                 diag_R_flag=True,
                 x=None, y=None, u=None)

        sub_pops = (np.arange(0, 11), np.arange(9, 20), np.arange(19, p))
        obs_pops = np.array((0,1,2))
        obs_time = prots[k]
        obs_scheme = ObservationScheme(p, T, sub_pops, obs_pops, obs_time)

        ###################
        #  generate data  #
        ###################

        truemodel = LDS(
            dynamics_distn=AutoRegression(A=pars_true['A'].copy(),sigma=pars_true['Q'].copy()),
            emission_distn=Regression_diag(A=np.hstack((pars_true['C'].copy(), pars_true['d'].copy().reshape(p,1))),
                                           sigma=pars_true['R'].copy(), affine=True),
                        )
        truemodel.mu_init = pars_true['mu0'].copy()
        truemodel.sigma_init = pars_true['V0'].copy()

        data, stateseq = truemodel.generate(T)

        ###################
        #    EM cycles    #
        ###################

        # get EM-step results from true parameters
        model = init_LDS_model(pars_true, data, obs_scheme) # reset to true pars
        model.E_step()
        stats_true,_ = collect_LDS_stats(model)
        model.M_step()   

        pars_init, _ = gen_pars(n, p, u_dim=0, 
                             pars_in=None, 
                             obs_scheme=obs_scheme,
                             gen_A='diagonal', lts=0.99 * np.ones((n,)),
                             gen_B='random', 
                             gen_Q='identity', 
                             gen_mu0='random', 
                             gen_V0='identity', 
                             gen_C='PCA', 
                             gen_d='mean', 
                             gen_R='fractionObserved',
                             diag_R_flag=True,
                             x=stateseq.T, y=data.T, u=None)

        # get EM-step results after m iterations                    
        model = init_LDS_model(pars_init, data, obs_scheme) # reset to initialisation                    
        print 'fitting'
        likes = [-np.infty]
        for i in progprint_xrange(m):
            likes.append(update(model))
            if np.abs(likes[-1] - likes[-2]) < eps or len(likes) > m:
                break
            
        stats_hat,pars_hat = collect_LDS_stats(model)

        ###################
        #  store results  #
        ###################

        from scipy.linalg import solve_discrete_lyapunov as dtlyap # solve discrete-time Lyapunov equation
        Pi   = dtlyap(pars_true['A'], pars_true['Q'])
        Pi_h = dtlyap(pars_hat['A'],  pars_hat['Q'])    
        Pi_t   = pars_true['A'].dot(Pi)
        Pi_t_h = pars_hat['A'].dot(Pi_h)

        save_file = '../../../results/cosyne_poster/simulation_1/' + 'k_' + str(k) + '_PCA_' + str(i)
        from scipy.io import savemat # store results for comparison with Matlab code   
        save_file_m = {'x': model.states_list[0].stateseq, 
                       'y': model.states_list[0].data,
                       'u' : [], 
                       'll' : likes, 
                       'T' : model.states_list[0].T, 
                       'Trial': len(model.states_list), 
                       'elapsedTime' : 0,
                       'ifUseB':False, 
                       'ifUseA':True, 
                       'epsilon':0,
                       'ifRDiagonal':False,
                       'covConvEps':0,        
                       'truePars':pars_true,
                       'initPars':pars_init,
                       'estPars': pars_hat,
                       'stats_h': stats_hat,
                       'stats_true': stats_true,
                       'Pi':     Pi,
                       'Pi_h' :  Pi_h,
                       'Pi_t':   Pi_t,
                       'Pi_t_h': Pi_t_h,
                       'obsScheme' : obs_scheme}

        savemat(save_file,save_file_m) # does the actual saving


        covy_h= np.dot( np.dot(pars_hat['C'], Pi_h),      pars_hat['C'].transpose()) + pars_hat['R']
        covy_t= np.dot(np.dot(pars_true['C'], Pi),pars_true['C'].transpose()) + np.diag(pars_true['R'])

        y_tl = np.zeros([2*p,T-1])
        y_tl[range(p),:] = data[range(0,T-1),:].T
        y_tl[range(p,2*p),:] = data[range(1,T),:].T
        covy = np.cov(y_tl)

        covy_e=    covy[np.ix_(range(p),range(p))]
        covy_tl_e= covy[np.ix_(range(p,2*p),range(0,p))]

        covy_tl_h= np.dot(np.dot(pars_hat['C'], np.dot(pars_hat['A'],Pi_h)), pars_hat['C'].transpose())
        covy_tl_t=(np.dot(np.dot(pars_true['C'],np.dot(pars_true['A'], Pi)),pars_true['C'].transpose()))
        #%matplotlib inline
        idx_stitched = np.ones([p,p],dtype = bool)
        for i in range(len(obs_scheme.sub_pops)):
            if len(obs_scheme.sub_pops[i])>0:
                idx_stitched[np.ix_(obs_scheme.sub_pops[i],obs_scheme.sub_pops[i])] = False
        plt.imshow(idx_stitched,interpolation='none')

        #%matplotlib inline
        fig = plt.figure(1, figsize=(20,10))
        plt.subplot(5,3,1)
        plt.plot(covy_e[np.invert(idx_stitched)], covy_t[np.invert(idx_stitched)], '.')
        plt.title('obs. emp vs. obs. true')
        plt.subplot(5,3,2)
        plt.plot(covy_e[np.invert(idx_stitched)], covy_h[np.invert(idx_stitched)], '.')
        plt.title('obs. emp vs. obs. stitched')
        plt.subplot(5,3,3)
        plt.plot(covy_t[np.invert(idx_stitched)], covy_h[np.invert(idx_stitched)], '.')
        plt.title('obs. true vs. obs. stitched')

        plt.subplot(5,3,4)
        plt.plot(covy_tl_e[np.invert(idx_stitched)], covy_tl_t[np.invert(idx_stitched)], '.')
        plt.title('obs. emp vs. non-obs. true')
        plt.subplot(5,3,5)
        plt.plot(covy_tl_e[np.invert(idx_stitched)], covy_tl_h[np.invert(idx_stitched)], '.')
        plt.title('obs. emp vs. non-obs. stitched')
        plt.subplot(5,3,6)
        plt.plot(covy_tl_t[np.invert(idx_stitched)], covy_tl_h[np.invert(idx_stitched)], '.')
        plt.title('obs. non-observed true vs. non-obs. stitched')

        plt.subplot(5,3,7)
        plt.plot(covy_e[idx_stitched], covy_t[idx_stitched], '.')
        plt.title('non-obs. emp vs. non-obs. true')
        plt.subplot(5,3,8)
        plt.plot(covy_e[idx_stitched], covy_h[idx_stitched], '.')
        plt.title('non-obs. emp vs. non-obs. stitched')
        plt.subplot(5,3,9)
        plt.plot(covy_t[idx_stitched], covy_h[idx_stitched], '.')
        plt.title('non-obs. true vs. non-obs. titched')

        plt.subplot(5,3,10)
        plt.plot(covy_tl_e[idx_stitched], covy_tl_t[idx_stitched], '.')
        plt.title('non-obs. emp vs. non-obs. true')
        plt.subplot(5,3,11)
        plt.plot(covy_tl_e[idx_stitched], covy_tl_h[idx_stitched], '.')
        plt.title('non-obs. emp vs. non-obs. stitched')
        plt.subplot(5,3,12)
        plt.plot(covy_tl_t[idx_stitched], covy_tl_h[idx_stitched], '.')
        plt.title('non-obs. true vs. non-obs. stitched')

        plt.subplot(5,3,13)
        plt.plot(likes[int(m//4):])
        plt.title('non-obs. true vs. non-obs. stitched')
        ax = fig.add_subplot(5,3,14)
        ax.text(0,0, 'final ll: ' + str(likes[-1]))

        plt.show()

